In [ ]:
import numpy as np
import nltk
import string
import random
import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import WordNetLemmatizer

# 데이터 로드 및 토큰화
with open("robots.txt", "r", encoding="utf-8", errors="ignore") as f:
    raw = f.read().lower()
sent_tokens = nltk.sent_tokenize(raw)

# Lemmatizer 초기화 및 함수 정의
lemmer = WordNetLemmatizer()


def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]


remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)


def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))


# 기존 텍스트 묶음을 미리 TF-IDF 벡터로 변환 (딱 한 번만 실행)
TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words="english")
tfidf_base = TfidfVec.fit_transform(sent_tokens)


# 챗봇 응답 함수: 사용자 입력을 외부에서 처리
def response(user_response):
    # 사용자 입력을 TfidfVectorizer에 'transform'만 적용
    tfidf_user = TfidfVec.transform([user_response])

    # 기존 텍스트 묶음과 사용자 입력 간의 코사인 유사도 계산
    vals = cosine_similarity(tfidf_base, tfidf_user)

    max_idx = np.argmax(vals)

    # 유사도 값이 0이면 이해하지 못했다고 응답
    if vals[max_idx][0] == 0:
        return "I am sorry! I don't understand you."
    else:
        # 가장 유사한 문장 반환
        return sent_tokens[max_idx]


# 시간 응답 함수
def tell_time(sentence):
    for word in sentence.split():
        if word.lower() == "time":
            currentdt = datetime.datetime.now()
            return currentdt.strftime("%Y-%m-%d %H:%M:%S")


# 인사말 응답을 위한 리스트
GREETING_INPUTS = ["hello", "hi", "greetings", "sup", "what's up", "hey", "hey there"]
GREETING_RESPONSES = [
    "hi",
    "hey",
    "*nods*",
    "hi there",
    "hello",
    "I am glad! You are talking to me",
]


# 인사말 함수
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)


# 메인 챗봇 루프
flag = True
print(
    "ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!"
)
while flag:
    user_response = input()
    user_response = user_response.lower()

    if user_response != "bye":
        if user_response == "thanks" or user_response == "thank you":
            flag = False
            print("ROBO: You are welcome..")
        else:
            if greeting(user_response):
                print("ROBO: " + greeting(user_response))
            elif tell_time(user_response):
                print("ROBO: " + tell_time(user_response))
            else:
                print("ROBO: ", end="")
                print(response(user_response))
    else:
        flag = False
        print("ROBO: Bye! take care..")

SyntaxError: invalid syntax (1541199660.py, line 53)